In [ ]:
MFCCS = 13
EMBEDDING_SIZE = 192

MODEL = "MFCC_ECAPA-TDNN_Genuine_Random"

LEARNING_RATE = 0.001
MARGIN = 1
NORM = 2

BATCH_SIZE = 16
EPOCHS = 2

In [ ]:
# Handle warnings
import warnings
warnings.filterwarnings("ignore")

# Logging to log file
import logging
logging.basicConfig(filename=f'../logs/{MODEL}.log',
                    level=logging.INFO,
                    format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# MLFlow configuration
import mlflow
mlflow.set_tracking_uri("../mlruns")
logging.getLogger('mlflow.utils.requirements_utils').setLevel(logging.ERROR)

# Get device
from utils import get_device, load_genuine_dataset, train_model_random_loss
device = get_device(logger)

# Imports
import torch.optim as optim
from torch.nn import TripletMarginLoss
from torch.utils.data import DataLoader
from dataloader import RandomTripletLossDataset, collate_triplet_fn
from frontend import MFCCTransform
from speechbrain.lobes.models.ECAPA_TDNN import ECAPA_TDNN

In [ ]:
train_labels, dev_labels, test_labels = load_genuine_dataset()

In [ ]:
audio_dataset = RandomTripletLossDataset(train_labels, frontend=MFCCTransform, logger=logger)
audio_dataloader = DataLoader(audio_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_triplet_fn)
model = ECAPA_TDNN(input_size=MFCCS, lin_neurons=EMBEDDING_SIZE, device=device)
model.to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
triplet_loss = TripletMarginLoss(margin=MARGIN, p=NORM)

In [ ]:
# Run training
train_model_random_loss(EPOCHS, audio_dataloader, model, triplet_loss, optimizer, device, MODEL, logger)